# Week 6 Lab Session
**MACS 30123** \
*Adam Wu, Wonje Yun*

# Preliminary

Start up your AWS Learner Lab, and remember to update your security keys in `~/.aws/credentials`.

In [1]:
import boto3
import json

# Part I: Review of Databases

## Exercise 1: S3 Databases

In this exercise, let's create a new AWS S3 Bucket and practice reading/writing files to it.

Recall that S3 is a "data lake" designed for generic storage of objects. For example you could store data, images, pre-trained models, etc. 

*Intuitively, you can think of a bucket on S3 as similar to Dropbox or some generic directory.*

**Note: The name of the bucket has to be unique on the AWS region.**

In [4]:
# Initialize boto3 handler
s3 = boto3.resource('s3')

# Create a new bucket to store your files
BUCKETNAME = 'adam-example-bucket'
s3.create_bucket(Bucket=BUCKETNAME)

# This is what we will use to interface with the specific bucket
bucket = s3.Bucket( BUCKETNAME )

s3.Bucket(name='adam-example-bucket')

You can go on the AWS website and verify that a new S3 bucket has been created. 

There is a provided `data/1_exampledata.json` file. Let's try uploading it to your S3 bucket. 

*Using the Dropbox analogy, you can think of `Key` as the name of the file, and `Body` as the raw contents of the file.*

In [5]:
# Read in your locally stored data
data = open('data/1_exampledata.json', 'rb')

# Upload to S3
FILENAME = 'file1.json'
bucket.put_object(Key=FILENAME, Body=data)

s3.Object(bucket_name='adam-example-bucket', key='file1.json')

In [29]:
# Get the keys in the bucket
print('Keys:', [obj.key for obj in bucket.objects.all()])

# Get `file1.json` from S3, parse it with json, and view it
obj = s3.Object(BUCKETNAME, 'file1.json')
body = json.loads(obj.get()['Body'].read())
body

Keys: ['file1.json']


{'user_id': '0001',
 'timestamp': '092821120000',
 'time_elapsed': '5',
 'q1': '5',
 'q2': '3',
 'q3': '2',
 'q4': '2',
 'q5': '4',
 'freetext': 'I had a very bad day today...'}

S3 charges you based on how much storage you use, though it's not much. Let's try deleting the bucket now.

*To delete a bucket, you must first empty the bucket by deleting everything inside it.*

In [9]:
# Delete each file/object in the bucket
for obj in bucket.objects.all():
    obj.delete()

print('Keys:', [obj.key for obj in bucket.objects.all()])

# Delete the bucket itself
bucket.delete()

Keys: []


{'ResponseMetadata': {'RequestId': '0BNTK43WB7HYZCXR',
  'HostId': 'mpHyPGhJqP+5pNvks18cGHfcLVdJjnZWS085gpbFw6df/gsGztcDBirzt/lgFFwVfJ7GFXYFZNY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'mpHyPGhJqP+5pNvks18cGHfcLVdJjnZWS085gpbFw6df/gsGztcDBirzt/lgFFwVfJ7GFXYFZNY=',
   'x-amz-request-id': '0BNTK43WB7HYZCXR',
   'date': 'Thu, 25 Apr 2024 00:57:58 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Exercise 2: DynamoDB Databases

In this exercise, let's create a new AWS DynamoDB database and practice reading/writing files to it.

Recall that DynamoDB is a "serverless" NoSQL database. Intuitively, you can think of DynamoDB as being distributed across multiple servers. This allows for high data availability. For example, if a traditional database hosted on a single server crashes, then you might lose your data. But this offers protection against that scenario, and also allows for it to easily scale as data increases.

To create a new table, the main things you need to supply are:
 - `TableName`: Name of the table
 - `KeySchema`: Describes the primary key and/or a secondary key
 - `AttributeDefinitions`: Describes the data types of the keys

It may be helpful to consult the documentation [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/service-resource/create_table.html#DynamoDB.ServiceResource.create_table).

In [11]:
# Initialize boto3 handler
ddb = boto3.resource('dynamodb')

DB_NAME = 'adam_example_db'

# Create a new DynamoDB Table
try:
    table = ddb.create_table(
        TableName=DB_NAME,
        KeySchema=[
            {
                'AttributeName': 'user_id',   # Here we define the primary key as `user_id`
                'KeyType': 'HASH'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'user_id',  # Now we describe what the data type of `user_id` is.
                'AttributeType': 'S'         # In this case, a string 'S'. In other cases, 'N' = number, 'B' = binary.
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,          # This specifies the maximum read/write capacity. Increasing it
            'WriteCapacityUnits': 1          # allows for it to scale, but can use up your AWS credits quickly
        }
    )
    table.meta.client.get_waiter('table_exists').wait(TableName=DB_NAME)
    
# Use the existing one if it already exists
except:
    table = ddb.Table(DB_NAME)

Now let's try putting our `1_exampledata.json` in there.

In [24]:
import json

# Read in our data
with open('data/1_exampledata.json', 'r') as f:
    data = json.load(f)
data

{'user_id': '0001',
 'timestamp': '092821120000',
 'time_elapsed': '5',
 'q1': '5',
 'q2': '3',
 'q3': '2',
 'q4': '2',
 'q5': '4',
 'freetext': 'I had a very bad day today...'}

In [25]:
# Insert our entry in the table
table.put_item(
   Item=data
)

{'ResponseMetadata': {'RequestId': 'QUGTMS8JBIUV3OEL3LFKR8FD3VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 25 Apr 2024 01:36:43 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'QUGTMS8JBIUV3OEL3LFKR8FD3VVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [31]:
# Get our entry from the table
response = table.get_item(
    Key={
        'user_id' : '0001'
    }
)
response

{'Item': {'q1': '5',
  'q2': '3',
  'user_id': '0001',
  'q3': '2',
  'q4': '2',
  'q5': '4',
  'timestamp': '092821120000',
  'freetext': 'I had a very bad day today...',
  'time_elapsed': '5'},
 'ResponseMetadata': {'RequestId': 'TS7OC4SQQRPNN1ELO59CKIEV6VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 25 Apr 2024 01:38:12 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '215',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'TS7OC4SQQRPNN1ELO59CKIEV6VVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3989283962'},
  'RetryAttempts': 0}}

The data we just loaded is in `response['Item']`, along with some HTTP response messages. `HTTPStatusCode = 200` means it was successful, and you may want to look up what these codes mean as a refresher.

As before, let's now delete the table.

In [32]:
table.delete()

{'TableDescription': {'TableName': 'adam_example_db',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:348752177325:table/adam_example_db',
  'TableId': '7b346ee6-f729-41fe-a50e-8a0768146d52',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'STVJJJT1F8GJO9IBV4FP2E1JM3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 25 Apr 2024 01:42:49 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '366',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'STVJJJT1F8GJO9IBV4FP2E1JM3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1675746389'},
  'RetryAttempts': 0}}

## Exercise 3: MySQL Databases

Next let's practice traditional relational databases with MySQL on AWS RDS. Unlike DynamoDB, this is hosted on a single EC2 instance.

In [34]:
# Initialize boto3 handler
rds = boto3.client('rds')

# Describes your currently active databases
DBs = rds.describe_db_instances()['DBInstances']
activeDBs = [ x['DBName'] for x in DBs ]
activeDBs

['adam_example_db']

*Tip: It can take some time to create the database. When debugging, you might want to just create it once and add some logic like:*

In [35]:
DB_NAME = 'adam_example_db'
INSTANCE_NAME = 'week6_lab'

if DB_NAME not in activeDBs:
    # Create a new MySQL database on AWS
    response = rds.create_db_instance(
        DBInstanceIdentifier=INSTANCE_NAME,
        DBName=DB_NAME,
        MasterUsername='username',
        MasterUserPassword='password',
        DBInstanceClass='db.t3.micro',   # This specifies the EC2 instance that will run the datbase
        Engine='MySQL',
        AllocatedStorage=5
    )
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier=INSTANCE_NAME)

As Jon mentioned in class, we need to adjust the security settings to be able to connect to it directly using our local machine. Be sure to include this in your assignments for example.

In [36]:
db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )

relational-db is available at relational-db.cccphtqtenb6.us-east-1.rds.amazonaws.com on Port 3306


In [37]:
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust security permissions
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Next let's connect to the database, create a new table, and add the example data.

In [38]:
import mysql.connector

# Connect to the MySQL database
conn =  mysql.connector.connect(host=ENDPOINT, 
                                user="username", 
                                passwd="password", 
                                port=PORT, 
                                database=DB_NAME)
cur = conn.cursor()

In [39]:
create_table = '''
               CREATE TABLE IF NOT EXISTS survey (
                   user_id VARCHAR(10),
                   timestamp INT,
                   time_elapsed INT,
                   q1 INT,
                   q2 INT,
                   q3 INT,
                   q4 INT,
                   q5 INT,
                   freetext TEXT,
                   PRIMARY KEY (user_id)
               )
               '''
insert_data  = '''
               INSERT INTO survey (user_id, timestamp, time_elapsed, q1, q2, q3, q4, q5, freetext)
               VALUES 
                   ('0001', 092821120000, 5, 5, 3, 2, 2, 4, 'I had a very bad day today...')
               '''

for op in [create_table, insert_data]:
    cur.execute(op)

In [40]:
# Query the table
cur.execute('SELECT * FROM survey')
query_results = cur.fetchall()
print(query_results)

[('0001', 2147483647, 5, 5, 3, 2, 2, 4, 'I had a very bad day today...')]


**Note: Since this runs on an EC2 instance, you should terminate it when you are done. This can be done by simply deleting the database.**

In [42]:
response = rds.delete_db_instance(DBInstanceIdentifier=INSTANCE_NAME,
                                     SkipFinalSnapshot=True
                                    )
status_db = response.get('ResponseMetadata').get('HTTPStatusCode')
status_db

200

# Part II: Assignment 2 Walkthrough

## Exercise 4: MySQL with Books

Now it's your turn. In `data/3_examplebook.json`, this is a sample of the data structure of a single book that you will scrape in Q2 in Assignment 2. 

Try to create a MySQL database, define the table based on this data structure, and insert it into the table. Work in groups if you are stuck.

In [ ]:
rds = boto3.client('rds')

## [TASK] Your code here

## Exercise 5: Lambda with Books

Now try writing a Lambda function which scrapes a batch of book pages, and inserts it into the database you just created.

Hint: Consult the serial code [here](https://github.com/Apress/practical-web-scraping-for-data-science/blob/master/books-to-scrape/scraper.py).

In [ ]:
import dataset

In [ ]:
event = [
    {
        'book_id' : 'a-light-in-the-attic_1000',
        'book_url' : 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/'
    },
    {
        'book_id' : 'tipping-the-velvet_999',
        'book_url' : 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/'
    }
]

def lambda_handler(event, context):
    """
    Task: Given an input `event`, which contains `book_id` and `book_url`,
          scrape the relevant url and insert it into the database you just created.
    """
    
    # Connect to your MySQL Database from Exercise 4
    db_url = f'mysql+mysqlconnector://{DB_USERNAME}:{DB_PASSWORD}@{ENDPOINT}:{PORT}/{DB_NAME}'
    db = dataset.connect(db_url)
    
    ## [TASK] Your code here
    batch_of_books_to_scrape = event
    
    return
    
lambda_handler(event=event, context=None)

After the code is working, zip it up in the provided `q2_deployment_package.zip` (on Canvas) and deploy it with boto3.

In [ ]:
lamb = boto3.client('lambda')

## [TASK] Your code here

## Exercise 6: SQS

A simple way to orchestrate your compute jobs is with AWS SQS (Simple Queue Service). 

In this exercise:
 1. Create an SQS queue using boto3 
 2. Configure it to trigger your Lambda function from Exercise 5
 3. Send `event1` and `event2` into the queue
 
Afterwards, check your MySQL database from Exercise 4 and see if everything is working as expected.

*Note: When you send a job to Lambda using SQS, the `event` input contains some metadata. You should now modify your Lambda function to parse inputs with:*

`batch_of_books_to_scrape = json.loads(event['Records'][0]['body'])`

In [ ]:
event1 = [
    {
        'book_id' : 'a-light-in-the-attic_1000',
        'book_url' : 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/'
    },
    {
        'book_id' : 'tipping-the-velvet_999',
        'book_url' : 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/'
    }
]

event2 = [
    {
        'book_id' : 'Soumission',
        'book_url' : 'http://books.toscrape.com/catalogue/soumission_998/'
    },
    {
        'book_id' : 'sharp-objects_997',
        'book_url' : 'http://books.toscrape.com/catalogue/sharp-objects_997/'
    }
]

In [ ]:
sqs = boto3.client('sqs')

## [TASK] Your code here

## Putting it all together

Read the Assignment 2 instructions carefully, and start thinking about how to approach it. If you are unsure how to start, try to break it down into steps and sketch it out on a piece of paper. 

For example:
1. First I will create some databases...
2. Then I will write a Lambda function that takes input `event` and does something...
3. In order to get `event`, I need to first get a list of book urls...
4. And so on